# LTI Advantage bootcamp notebook

The notebook shows how to interact with the LTI Advantage ecosystem. It interacts with an actual server which is used to exercise the service APIs. 


## Setup

In [22]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'

In [45]:
# First we need to get a new tool deployment from the server for this notebook instance
# Each tool must have a client_id and a private key, and the location of the platform
# keyset to be able to interact with the platform.
# How this information is obtained by the tool is NOT currently part of the LTI specifications.

r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)
print(tool_info)

{'client_id': '0', 'deployment_id': 'deployment_0', 'key': {'e': 'AQAB', 'kty': 'RSA', 'use': 'sig', 'alg': 'RS256', 'd': 'EtlFjk-0MRjRqAg7FLrIuRYHA0XbcewuIWl_Eu6qcPIwDn3psiyYCBpTCgR2vlTmTUrRNErsUCUrtCpP007l5Lv_Nc5ZAUVM5qAARMAeG_8RnR1bksc5tKkpsS0nQa0ZpjnemtsUpxeuHW4QK7Xa-JWfE9wIR1HXljFrpNp4BrenCFSi0jwTi3fRYoY2Px58ZWjvn8jTXOGJj5XN1y9kdNAAr8jbIVWh9AL2EXcyoPa1cVrUxMaB7dJ302VvGm57d_xtwlRp8FI1xnQfEUOA0y_Py-PohuGAIHuCaUZlvJynA6swSPFuFFA18A8ywvEURDuUx1GNiju1wTgoDA0h'}}


## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

In [46]:

r = requests.get("{}/tool/{}/cisr".format(platform_url, "0"))
post_data = {
    'id_token':r.text
}
print(post_data)

{'id_token': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IjE1MTcwODU0ODVfMCJ9.eyJub25jZSI6IjFiMTRkNWMwLTAzYTItMTFlOC1hMjJjLWM0OGU4ZmZiNzg1NyIsImlhdCI6MTUxNzA4NTU0MCwibmFtZSI6IkViYmEgRWxpc2EiLCJodHRwOi8vaW1zZ2xvYmFsLm9yZy9sdGkvdmVyc2lvbiI6IjEuMy4wIiwiZXhwIjoxNTE3MDg1NjAwLCJmYW1pbHlfbmFtZSI6IkVsaXNhIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL2RlcGxveW1lbnRfaWQiOiJkZXBsb3ltZW50XzAiLCJzdWIiOiJMVElCQ1VfMTAiLCJhdWQiOiIwIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL21lc3NhZ2VfdHlwZSI6IkNvbnRlbnRJdGVuU2VsZWN0aW9uUmVxdWVzdCIsImlzcyI6Imh0dHA6Ly9sb2NhbGhvc3Q6NTAwMCIsImdpdmVuX25hbWUiOiJFYmJhIiwiZW1haWwiOiJFYmJhLkVsaXNhQGV4YW1wbGUuY29tIiwiaHR0cDovL2ltc2dsb2JhbC5vcmcvbHRpL3Rvb2xfcGxhdGZvcm0iOnsiZ3VpZCI6Imx0aWJjX2F0XzE1MTcwODU0ODYiLCJuYW1lIjoiTFRJIEJvb3RjYW1wIFBsYXRmb3JtIn19.Ux360kx84XWvtnATdluVMnj49NzLUzL7Fg8jiQl6u4yFbg2BoFryQSIkra0ttRElzabk5Ce5n_nYewytfuZbKBfh27CW5xuOOl0wQmHLKtkTrdUbsmO-y2LFOinVPiwXZF5DyJ0UkpOwn6zM2nPzriSw2QRuuev0iASa744BBapbsSU8YYr0rzVGHsJIeZlokHGd7F_Z_Jf5xJR3SGfbcW5rGXZD7-BM_2rTpn6n4b77HiVgwQpF

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request. This request will come as a FORM post from the browser as an signed JWT, which is here simulated the post_data object.

### Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.
The platform exposes its keys in a keyset format at a well-know location (.well-known/)

In [47]:
# We need the key set now
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

print(keyset)
platform_keys = {}
# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    
print(platform_keys)

{'keys': [{'e': 'AQAB', 'use': 'sig', 'alg': 'HS256', 'kty': 'rsa', 'n': 'rIPY8sXpyABo1aUSA6mtJlIy5vocedxpdMXu5wYk-v1SzM4HnuIyedwm14wzoSr-tK61_4Wm125cg-d7ON0uYx6l6wI13LOPzSY96EaNarFF2JYmnywFWIYUsxlK24zP1YAe0AB8DMDLhbK1TDjXKHbryxx7J6nARa7RFLBqV-pKb3kwiMAPzUjbOrU_NL60Zuj68xHLvfJ2gIP2yzs_4rys3lLpDYEY9CzvyeDAwTMC3yF9hC_AE9RGg7g31wuFHMZPCL4mxEnLBl2xXIrnDxPilHh8ICeZEoVgEZOjUTItbdfls7XpiLE491VKezSLhqibByXH9VvOgdhcQRDSwQ==', 'kid': '1517085485_0'}, {'e': 'AQAB', 'use': 'sig', 'alg': 'HS256', 'kty': 'rsa', 'n': '02VIow7RzJWtceXlnyI9B0TYzJj_2mAM4SgSPux50RfO7N7fpnyMn7bnRsYE7RjYi0fdl__R1MpZtGy73dmZdIlVNF0HWd7tKdgyoboW3iqC_nwztXaTk8fSHnqfUwEdivoVcxDDok2tdkd1QjoYgijkfsBL6SwZuLiIjkMmPA_72jhs4AwFMdrGk-D3xeyuSjRDxVp3fpBksKnQdczLz9lOkYfSaKB0hGqQcdkuxNrOrNQ4m_JivGelSR_umj8qIHJS6xOfZEjxmObsydC5M0R20Aq2l92F_8yFS0t57tJ95cMNjnmHrfmCWvtcqalHytUQ8VU0jvf5g-TkDB-ILw==', 'kid': '1517085485_1'}, {'e': 'AQAB', 'use': 'sig', 'alg': 'HS256', 'kty': 'rsa', 'n': '0XHEnjjUbB-oPw_GVxbawO9Ku6brxxzcLhZFiv4WbB67Gjmefs0RtVEY

In [48]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

print(content_item_message)

{'http://imsglobal.org/lti/version': '1.3.0', 'iss': 'http://localhost:5000', 'sub': 'LTIBCU_10', 'given_name': 'Ebba', 'email': 'Ebba.Elisa@example.com', 'family_name': 'Elisa', 'iat': 1517085540, 'http://imsglobal.org/lti/message_type': 'ContentItenSelectionRequest', 'nonce': '1b14d5c0-03a2-11e8-a22c-c48e8ffb7857', 'http://imsglobal.org/lti/deployment_id': 'deployment_0', 'aud': '0', 'exp': 1517085600, 'http://imsglobal.org/lti/tool_platform': {'guid': 'ltibc_at_1517085486', 'name': 'LTI Bootcamp Platform'}, 'name': 'Ebba Elisa'}
